<a href="https://colab.research.google.com/github/KunalVaidya99/Image-Colorization-Using-Deep-Learning/blob/main/ChromaGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/5wErqiQHCitTE68WhpVMh6UbS2PuPLfWezYydwDKREoms0b94esL8TY


In [ ]:
import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import (Conv2D,Conv2DTranspose,UpSampling2D,MaxPooling2D,Input
                          ,Activation,Concatenate,BatchNormalization)
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import plot_model
from keras.models import Model
import numpy as np
from keras.preprocessing.image import load_img
from PIL import Image
import os
import random
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.applications import VGG16
from keras.preprocessing.image import load_img,img_to_array

Using TensorFlow backend.


Extracting Features for all images using VGG16

In [ ]:
def extract_features(directory):

  model = VGG16(include_top=True,weights="imagenet",classes=1000)
  features = dict()
  count = 0
  for filename in os.listdir(directory):
   
    image = load_img(directory+filename,target_size=(224,224))
    img_array = img_to_array(image)
    img_array = img_array.reshape((1,224,224,3))
    feature = model.predict(img_array)
    features[filename] = feature
    count = count + 1

    if (count%1000==0):
      print("Completed Feature Extraction Of " + f"{count}") 

  return features

directory = "/content/gdrive/My Drive/val_256/"
features = extract_features(directory)       


Completed Feature Extraction Of 1000
Completed Feature Extraction Of 2000
Completed Feature Extraction Of 3000
Completed Feature Extraction Of 4000
Completed Feature Extraction Of 5000
Completed Feature Extraction Of 6000
Completed Feature Extraction Of 7000
Completed Feature Extraction Of 8000
Completed Feature Extraction Of 9000
Completed Feature Extraction Of 10000
Completed Feature Extraction Of 11000
Completed Feature Extraction Of 12000
Completed Feature Extraction Of 13000
Completed Feature Extraction Of 14000
Completed Feature Extraction Of 15000
Completed Feature Extraction Of 16000
Completed Feature Extraction Of 17000
Completed Feature Extraction Of 18000
Completed Feature Extraction Of 19000
Completed Feature Extraction Of 20000
Completed Feature Extraction Of 21000
Completed Feature Extraction Of 22000
Completed Feature Extraction Of 23000
Completed Feature Extraction Of 24000
Completed Feature Extraction Of 25000
Completed Feature Extraction Of 26000
Completed Feature Ex

In [ ]:
import pickle
f = open("/content/gdrive/My Drive/ImageColorization /chromaVGG16.pkl","wb")
pickle.dump(features,f)
f.close()

UnsupportedOperation: ignored

In [ ]:
f_new = open("/content/gdrive/My Drive/ImageColorization /chromaVGG16.pkl","rb")
features = pickle.load(f_new)
f_new.close()

EOFError: ignored

In [ ]:
class DataGenerator():

  def __init__(self,features,directory,dim=(224,224),batch_size=32,n_channels=3):
    
    self.directory = directory
    self.dim = dim
    self.batch_size=batch_size
    self.n_channels = n_channels
    self.features = features
    self.epoch_end()

  def epoch_end(self,filenames):
    np.random.shuffle(filenames)

  def generate_batch(self,step_number,filenames):
    

    batch_filenames = filenames[step_number*self.batch_size:(step_number+1)*self.batch_size]

    gray_image = np.empty((self.batch_size,*self.dim,self.n_channels))
    color_image = np.empty((self.batch_size,*self.dim,self.n_channels))
    VGG16_target = np.empty((self.batch_size,1000,1))
    count = 0


    for filename in batch_filenames:
      image = Image.open(self.directory +"/" + filename,target_size=(224,224))
      image = image.resize((224,224))


      if (np.array(image).shape==(224,224,3)):
        
        gray_image[count] = (np.expand_dims(np.array(image.convert("L")),axis=-1))/255
        color_image[count] = np.array(image)/255
        VGG16_target = self.features[filename]

    return gray_image,color_image,VGG16_target    

In [ ]:
def patch_discriminator(img_shape):

  input_img = Input(shape=img_shape)

  X = Conv2D(filters=64,kernel_size=(4,4),strides=(2,2),padding='same')(input_img)
  X = BatchNormalization(axis=-1)(X)
  X = LeakyReLU(alpha=0.2)(X)

  X = Conv2D(filters=128,kernel_size=(4,4),strides=(2,2),padding='same')(X)
  X = BatchNormalization(axis=-1)(X)
  X = LeakyReLU(alpha=0.2)(X)

  X = Conv2D(filters=256,kernel_size=(4,4),strides=(2,2),padding='same')(X)
  X = BatchNormalization(axis=-1)(X)
  X = LeakyReLU(alpha=0.2)(X)

  X = Conv2D(filters=512,kernel_size=(4,4),strides=(1,1),padding='same')(X)
  X = BatchNormalization(axis=-1)(X)
  X = LeakyReLU(alpha=0.2)(X)

  X = Conv2D(filters=1,kernel_size=(4,4),strides=(1,1),padding='same')(X)
  X = BatchNormalization(axis=-1)(X)
  X = LeakyReLU(alpha=-1)(X)

  model = Model(inputs=input_img,outputs=X)

  return model


In [ ]:
patch_discriminator = patch_discriminator((224,224,3))
patch_discriminator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 112, 112, 64)      3136      
_________________________________________________________________
batch_normalization_6 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 56, 56, 128)       131200    
_________________________________________________________________
batch_normalization_7 (Batch (None, 56, 56, 128)       512       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 56, 56, 128)       0   

In [ ]:
 model = VGG16(include_top=False,weights="imagenet",classes=1000)

In [ ]:
def generator(img_shape):

  input_img = Input(shape=img_shape)
  X = input_img

  for layer in model.layers[:-6]:
    X = layer(X)
  
  VGG16_output = X 
  X = Conv2D(filters=512,kernel_size=(3,3),strides=(2,2),padding="same")(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)

  X = Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding="same")(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)

  X = Conv2D(filters=512,kernel_size=(3,3),strides=(2,2),padding="same")(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)

  X = Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding="same")(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)

  color_distribution = Flatten()(X)
  color_distribution = Dense(units=4096)
  color_distribution = Dense(units=4096)
  color_distribution = Dense(units=1000,activation='softmax')

  fusion = Flatten()(X)
  fusion = Dense(units=1024)
  fusion = Dense(units=512)
  fusion = Dense(units=256)
  fusion = RepeatVector(28*28)(fusion)

  encoder_output = Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding="same")(VGG16_output)
  encoder_output = BatchNormalization(axis=-1)(encoder_output)
  encoder_output = Activation('relu')(X)
  encoder_output = Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding="same")(VGG16_output)
  encoder_output = BatchNormalization(axis=-1)(encoder_output)
  encoder_output = Activation('relu')(X)

  encoder_output = Concatenate(axis=-1)([encoder_output,fusion])

  decoder = Conv2D(filters=256,kernel_size=(1,1),strides=(1,1),activation='relu',padding='same')(encoder_output)
  decoder = Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(decoder)
  decoder = UpSampling2D((2,2))(decoder)

  decoder = Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(decoder)
  decoder = Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(decoder)
  decoder = UpSampling2D((2,2))(decoder)

  decoder = Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(decoder)
  decoder = Conv2D(filters=3,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(decoder)
  decoder = UpSampling2D((2,2))(decoder)


  model = Model(inputs=input_img,outputs=[decoder,color_distribution])

  return model